In [78]:
from sympy.physics.vector import dynamicsymbols
from sympy.physics.vector import init_vprinting
init_vprinting()
from sympy import *

In [79]:
q1,q2,q3=dynamicsymbols('q1,q2,q3')
u1,u2,u3=dynamicsymbols('u1,u2,u3')
u1d,u2d,u3d=dynamicsymbols('u1d,u2d,u3d')
L_1,L_2=symbols('L_1,L_2')
F1,F2=symbols('F1,F2')
m1,m2,I1,I2,g=symbols('m1,m2,I1,I2,g')

 ## List of symbols ##

In [80]:
brot=[q1,q2]
brotd=[u1,u2]
brotd2=[u1d,u2d]

In [81]:
brot,brotd,brotd2

In [82]:
PQ=[[L_1,0],[L_2,0]]
PG=[[L_1,0],[L_2,0]]
Mass=[m1,m2]
Inertia=[0,0]
F_app=[[m1*g,0],[m2*g,0]]
M_app=[0,0]

## List of matrices ##

In [83]:
shifterloc=[]
q_r=Matrix([])
for i in range(0,len(brot)):
    sloc=Matrix([[cos(brot[i]),sin(brot[i])],[-sin(brot[i]),cos(brot[i])]])
    shifterloc.append(sloc)

In [84]:
shifterloc

⎡⎡cos(q₁)   sin(q₁)⎤  ⎡cos(q₂)   sin(q₂)⎤⎤
⎢⎢                 ⎥, ⎢                 ⎥⎥
⎣⎣-sin(q₁)  cos(q₁)⎦  ⎣-sin(q₂)  cos(q₂)⎦⎦

In [85]:
shifter=[None]*len(brot)
for i in range(0,len(brot)):
    sprev=eye(2)
    for j in range(0,len(brot)):
        sprev=sprev*shifterloc[j]
        #sprev=shifterloc[j]
        shifter[j]=sprev
        shifter[j].simplify()

In [86]:
shifter

⎡⎡cos(q₁)   sin(q₁)⎤  ⎡cos(q₁ + q₂)   sin(q₁ + q₂)⎤⎤
⎢⎢                 ⎥, ⎢                           ⎥⎥
⎣⎣-sin(q₁)  cos(q₁)⎦  ⎣-sin(q₁ + q₂)  cos(q₁ + q₂)⎦⎦

In [87]:
omega=[None]*len(brot)
alpha=[None]*len(brot)
for i in range(0,len(brot)):
    omega[i]=diff(shifter[i])*shifter[i].transpose()
    omega[i].simplify()
    omega[i]=omega[i].subs([(diff(brot[0]),brotd[0]),(diff(brot[1]),brotd[1])])
    alpha[i]=diff(omega[i])
    alpha[i]=alpha[i].subs([(diff(brotd[0]),brotd2[0]),(diff(brotd[1]),brotd2[1])])

In [88]:
omega

⎡⎡ 0   u₁⎤  ⎡   0      u₁ + u₂⎤⎤
⎢⎢       ⎥, ⎢                 ⎥⎥
⎣⎣-u₁  0 ⎦  ⎣-u₁ - u₂     0   ⎦⎦

In [89]:
alpha

⎡⎡ 0    u1d⎤  ⎡    0       u1d + u2d⎤⎤
⎢⎢         ⎥, ⎢                     ⎥⎥
⎣⎣-u1d   0 ⎦  ⎣-u1d - u2d      0    ⎦⎦

In [90]:
Twq=eye(len(brot))
for i in range(1,len(brot)):
    for j in range(0,i):
        Twq[j,i]=1

In [91]:
Wpd=Twq

In [92]:
Wpd

⎡1  1⎤
⎢    ⎥
⎣0  1⎦

In [93]:
S_r=[]
S_p=[]
body_r=[]
body_p=[]
for i in range(0,len(brot)):
    S_r.append(Matrix([-PG[i][1],PG[i][0]]).T)
    S_p.append(Matrix([-PQ[i][1],PQ[i][0]]).T)

In [94]:
S_r

[[0  L₁], [0  L₂]]

In [95]:
S_p

[[0  L₁], [0  L₂]]

## Full matrices ##

In [96]:
V1=zeros(len(brot),2*len(brot))
for j in range(0,len(brot)):
    for i in range(0,j):
        V1[i,j*2:j*2+2]=S_p[i]*shifter[i]
    for i in range(j,j+1):
        V1[i,j*2:j*2+2]=S_r[i]*shifter[i]

In [97]:
V1

⎡-L₁⋅sin(q₁)  L₁⋅cos(q₁)    -L₁⋅sin(q₁)       L₁⋅cos(q₁)   ⎤
⎢                                                          ⎥
⎣     0           0       -L₂⋅sin(q₁ + q₂)  L₂⋅cos(q₁ + q₂)⎦

In [98]:
Vpd=Twq*V1

In [99]:
Vpd

⎡-L₁⋅sin(q₁)  L₁⋅cos(q₁)  -L₁⋅sin(q₁) - L₂⋅sin(q₁ + q₂)  L₁⋅cos(q₁) + L₂⋅cos(q
⎢                                                                             
⎣     0           0             -L₂⋅sin(q₁ + q₂)               L₂⋅cos(q₁ + q₂)

₁ + q₂)⎤
       ⎥
       ⎦

In [100]:
mat1=zeros(len(brot),2*len(brot))
for j in range(0,len(brot)):
    for i in range(0,j):
        mat1[i,j*2:j*2+2]=S_p[i]*omega[i]*shifter[i]*Mass[j]*eye(2)
    for i in range(j,j+1):
        mat1[i,j*2:j*2+2]=S_r[i]*omega[i]*shifter[i]*Mass[j]*eye(2)

mat1

⎡-L₁⋅m₁⋅u₁⋅cos(q₁)  -L₁⋅m₁⋅u₁⋅sin(q₁)        -L₁⋅m₂⋅u₁⋅cos(q₁)              -L
⎢                                                                             
⎣        0                  0          L₂⋅m₂⋅(-u₁ - u₂)⋅cos(q₁ + q₂)  L₂⋅m₂⋅(-

₁⋅m₂⋅u₁⋅sin(q₁)      ⎤
                     ⎥
u₁ - u₂)⋅sin(q₁ + q₂)⎦

In [101]:
A1=Twq*mat1
A1

⎡-L₁⋅m₁⋅u₁⋅cos(q₁)  -L₁⋅m₁⋅u₁⋅sin(q₁)  -L₁⋅m₂⋅u₁⋅cos(q₁) + L₂⋅m₂⋅(-u₁ - u₂)⋅co
⎢                                                                             
⎣        0                  0                    L₂⋅m₂⋅(-u₁ - u₂)⋅cos(q₁ + q₂)

s(q₁ + q₂)  -L₁⋅m₂⋅u₁⋅sin(q₁) + L₂⋅m₂⋅(-u₁ - u₂)⋅sin(q₁ + q₂)⎤
                                                             ⎥
                      L₂⋅m₂⋅(-u₁ - u₂)⋅sin(q₁ + q₂)          ⎦

In [102]:
mat2=zeros(len(brot),2*len(brot))
for j in range(0,len(brot)):
    for i in range(0,j):
        mat2[i,j*2:j*2+2]=S_p[i]*shifter[i]*Mass[j]*eye(2)
    for i in range(j,j+1):
        mat2[i,j*2:j*2+2]=S_r[i]*shifter[i]*Mass[j]*eye(2)

mat2

⎡-L₁⋅m₁⋅sin(q₁)  L₁⋅m₁⋅cos(q₁)    -L₁⋅m₂⋅sin(q₁)       L₁⋅m₂⋅cos(q₁)   ⎤
⎢                                                                      ⎥
⎣      0               0        -L₂⋅m₂⋅sin(q₁ + q₂)  L₂⋅m₂⋅cos(q₁ + q₂)⎦

In [103]:
A2=Twq*mat2
A2

⎡-L₁⋅m₁⋅sin(q₁)  L₁⋅m₁⋅cos(q₁)  -L₁⋅m₂⋅sin(q₁) - L₂⋅m₂⋅sin(q₁ + q₂)  L₁⋅m₂⋅cos
⎢                                                                             
⎣      0               0                -L₂⋅m₂⋅sin(q₁ + q₂)                  L

(q₁) + L₂⋅m₂⋅cos(q₁ + q₂)⎤
                         ⎥
₂⋅m₂⋅cos(q₁ + q₂)        ⎦

In [104]:
A3=zeros(len(brot),len(brot))
for j in range(0,len(brot)):
    for i in range(0,len(brot)):
        A3[i,j]=Twq[i,j]*Inertia[j]
A3

⎡0  0⎤
⎢    ⎥
⎣0  0⎦

## Column vectors of gen vel, active force and active moment ##

In [105]:
qdvec = Matrix([])
for i in range(0,len(brot)):
    qdvec = qdvec.col_join(Matrix([brotd[i]]))

In [106]:
qdvec

⎡u₁⎤
⎢  ⎥
⎣u₂⎦

In [107]:
F_active = Matrix([])
for i in range(0,len(brot)):
    for j in range(0,2):
        F_active = F_active.col_join(Matrix([F_app[i][j]]))

In [108]:
F_active

⎡g⋅m₁⎤
⎢    ⎥
⎢ 0  ⎥
⎢    ⎥
⎢g⋅m₂⎥
⎢    ⎥
⎣ 0  ⎦

In [109]:
M_active = Matrix([])
for i in range(0,len(brot)):
    M_active = M_active.col_join(Matrix([M_app[i]]))

In [110]:
M_active

⎡0⎤
⎢ ⎥
⎣0⎦

In [111]:
f1 = Vpd*F_active + Wpd*M_active

In [112]:
f1.simplify()

In [113]:
f=f1-Vpd*A1.T*qdvec

In [114]:
f.simplify()
f

⎡                                      2                                      
⎢2⋅L₁⋅L₂⋅m₂⋅u₁⋅u₂⋅sin(q₂) + L₁⋅L₂⋅m₂⋅u₂ ⋅sin(q₂) - L₁⋅g⋅m₁⋅sin(q₁) - L₁⋅g⋅m₂⋅s
⎢                                                                             
⎢                                        ⎛     2                         ⎞    
⎣                                 -L₂⋅m₂⋅⎝L₁⋅u₁ ⋅sin(q₂) + g⋅sin(q₁ + q₂)⎠    

                             ⎤
in(q₁) - L₂⋅g⋅m₂⋅sin(q₁ + q₂)⎥
                             ⎥
                             ⎥
                             ⎦

In [115]:
Mass = Vpd*A2.T + Wpd*A3.T

In [116]:
Mass.simplify()

In [117]:
Mass

⎡  2        2                             2                            ⎤
⎢L₁ ⋅m₁ + L₁ ⋅m₂ + 2⋅L₁⋅L₂⋅m₂⋅cos(q₂) + L₂ ⋅m₂  L₂⋅m₂⋅(L₁⋅cos(q₂) + L₂)⎥
⎢                                                                      ⎥
⎢                                                         2            ⎥
⎣           L₂⋅m₂⋅(L₁⋅cos(q₂) + L₂)                     L₂ ⋅m₂         ⎦